## > Setup for all Labs

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## > Initial setup Lab01

In [4]:
from utils import (
    upload_file_to_s3,
)

import sagemaker
import os
import time

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()  # execution role for the endpoint
region = sagemaker_session._region_name

bucket = sagemaker_session.default_bucket()
prefix = "swagger_codegen"

# claude pricing in us-east-1 pricing
input_per_1k = 0.00025
output_per_1k = 0.00125

data_dir = "../data"
yml_dir = f"{data_dir}/yml_files"
uml_dir = f"{data_dir}/uml_diagrams"

### > Upload data to S3 to setup the lab

In [2]:
data_dirs= [yml_dir,
            uml_dir,
            f"{data_dir}/yml_questions",
            f"{data_dir}/uml_questions"]

for ddir in data_dirs:
    for filename in os.listdir(ddir):
        filepath = os.path.join(ddir, filename)

        # yml upload file to s3
        key = f"{prefix}/{filepath.replace(data_dir+'/', '')}"
        s3_path = upload_file_to_s3(filepath, bucket, key)

    print(f"Sync data to S3 from {ddir} =========")

File '../data/yml_files/uspto.yml' uploaded successfully to s3://sagemaker-us-east-1-699946278336/swagger_codegen/yml_files/uspto.yml
File '../data/yml_files/petstore.yml' uploaded successfully to s3://sagemaker-us-east-1-699946278336/swagger_codegen/yml_files/petstore.yml
File '../data/yml_files/flowerstore.yml' uploaded successfully to s3://sagemaker-us-east-1-699946278336/swagger_codegen/yml_files/flowerstore.yml
File '../data/yml_files/bookstore.yml' uploaded successfully to s3://sagemaker-us-east-1-699946278336/swagger_codegen/yml_files/bookstore.yml
File '../data/yml_files/link.yml' uploaded successfully to s3://sagemaker-us-east-1-699946278336/swagger_codegen/yml_files/link.yml
Sync data to S3 from ../data/yml_files =========
File '../data/uml_diagrams/bookstore.jpg' uploaded successfully to s3://sagemaker-us-east-1-699946278336/swagger_codegen/uml_diagrams/bookstore.jpg
File '../data/uml_diagrams/link.jpg' uploaded successfully to s3://sagemaker-us-east-1-699946278336/swagger_c

### > Store the parameter for future labs

In [3]:
%store bucket
%store prefix
%store yml_dir
%store uml_dir
%store data_dir
print(bucket)

Stored 'bucket' (str)
Stored 'prefix' (str)
Stored 'yml_dir' (str)
Stored 'uml_dir' (str)
Stored 'data_dir' (str)
sagemaker-us-east-1-699946278336


## > Initial Setup Lab02


In [15]:
import boto3
import pprint as pp


from utility_2 import (
    create_bedrock_execution_role,
    create_policies_in_oss,
    create_oss_policy_attach_bedrock_execution_role
)

boto3_session = boto3.Session()
region_name = boto3_session.region_name
aoss_client = boto3_session.client('opensearchserverless')

vector_store_name = 'swagger-api-vector'

exists = aoss_client.list_collections(
    collectionFilters={'name': vector_store_name},
    maxResults=10
)

if len(exists['collectionSummaries']) == 0:
    
    bedrock_kb_execution_role = create_bedrock_execution_role(bucket_name=bucket)
    bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']
    
    # create security, network and data access policies within OSS
    encryption_policy, network_policy, access_policy = create_policies_in_oss(vector_store_name=vector_store_name,
                       aoss_client=aoss_client,
                       bedrock_kb_execution_role_arn=bedrock_kb_execution_role_arn)


    print(f"creating vector collection {vector_store_name}")
    vector_collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')
    vector_collection_arn = vector_collection["createCollectionDetail"]['arn']
    vector_collection_id = vector_collection['createCollectionDetail']['id']
    
    pp.pprint(vector_collection)
    time.sleep(10)
    
    # # create oss policy and attach it to Bedrock execution role
    # create_oss_policy_attach_bedrock_execution_role(collection_id=vector_collection_id,
    #                                             bedrock_kb_execution_role=bedrock_kb_execution_role)

else:
    print(f"vector collection {vector_store_name} exists")
    vector_collection_arn = exists['collectionSummaries'][0]['arn']
    vector_collection_id = exists['collectionSummaries'][0]['id']

vector_host = vector_collection_id + '.' + region_name + '.aoss.amazonaws.com'
print(vector_host)

%store vector_store_name
%store vector_collection_arn
%store vector_collection_id
%store vector_host


vector collection vector-content exists
phlcxhubrgldq0vs7upf.us-east-1.aoss.amazonaws.com
Stored 'vector_store_name' (str)
Stored 'vector_collection_arn' (str)
Stored 'vector_collection_id' (str)
Stored 'vector_host' (str)
